## The Spectrum of Crime
This visualization tool provides an interactive exploration of the NYPD Arrest and Hate Crime dataset, which contains detailed records of incidents from 2019 to 2023. By dynamically comparing the number of arrests and hate crimes over time, and providing detailed breakdowns by borough and offense category, this tool can help researchers, policy makers, and the public better understand crime trends in New York City, and potentially inform strategies for crime prevention and community safety.

### Visualizing NYPD Arrest Data
The graph in this code is a scatter mapbox plot created using Plotly Express. It visualizes crime data on a map of New York City, with each point representing a crime incident. The points are colored based on the type of offense, and the data displayed can be filtered by age, borough, and sex using dropdown menus.

In [1]:
import dash
from dash import dcc, html
import plotly.express as px
import pandas as pd

# Load the dataset
df = pd.read_csv("NYPD_Arrest_Data.csv")

# Filter and map categories
category_map = {'F': 'Felony', 'M': 'Misdemeanor', 'V': 'Violation'}
df = df[df['LAW_CAT_CD'].isin(category_map.keys())]  # Filter the data to only include rows with keys in category_map
df['LAW_CAT_CD'] = df['LAW_CAT_CD'].map(category_map)  # Map the categories to their full names

# Map for full borough names
borough_full_names = {'B': 'Bronx', 'K': 'Brooklyn', 'M': 'Manhattan', 'Q': 'Queens', 'S': 'Staten Island'}
df['ARREST_BORO'] = df['ARREST_BORO'].map(borough_full_names)  # Map the borough codes to their full names

# Rename columns for better hover data labels
df.rename(columns={
    'LAW_CAT_CD': 'Offense',
    'PERP_SEX': 'Sex',
    'ARREST_BORO': 'Borough',
    'AGE_GROUP': 'Age'
}, inplace=True)  # Rename columns for better readability

# Define the order of age groups
age_order = ['<18', '18-24', '25-44', '45-64', '65+']

# Add a new column for color
color_map = {'Felony': '#58394E', 'Misdemeanor': '#6887AA', 'Violation': '#4EC4D3'}
df['Color'] = df['Offense'].map(color_map)  # Map the offenses to their corresponding colors

# Initialize the Dash app
external_stylesheets = ['https://fonts.googleapis.com/css2?family=Roboto&display=swap']
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)  # Initialize the Dash app with external stylesheets

# Define the layout of the app
app.layout = html.Div([
    html.H1("NYPD Arrest Data Visualization", style={'color': 'white', 'textAlign': 'center', 'font-family': 'Roboto', 'fontSize': '24px'}),
    html.Div([
        html.Div([
            # Define the age dropdown menu
            html.Div([
                html.Label('Age:', style={'color': 'white', 'fontWeight': 'bold', 'fontSize': '16px', 'font-family': 'Roboto'}),
                dcc.Dropdown(
                id='age-dropdown',
                options=[{'label': age, 'value': age} for age in age_order],
                value=None,
                style={'backgroundColor': 'white', 'color': 'black'}
            ),
            ], style={'width': '20%', 'display': 'inline-block', 'padding': '10px', 'backgroundColor': 'rgba(0,0,0,0)'}),

            # Define the borough dropdown menu
            html.Div([
                html.Label('Borough:', style={'color': 'white', 'fontWeight': 'bold', 'fontSize': '16px', 'font-family': 'Roboto'}),
                dcc.Dropdown(
                id='borough-dropdown',
                options=[{'label': borough, 'value': borough} for borough in df['Borough'].unique()],
                value=None,
                style={'backgroundColor': 'white', 'color': 'black'}
            ),
            ], style={'width': '20%', 'display': 'inline-block', 'padding': '10px', 'backgroundColor': 'rgba(0,0,0,0)'}),

            # Define the sex dropdown menu
            html.Div([
                html.Label('Sex:', style={'color': 'white', 'fontWeight': 'bold', 'fontSize': '16px', 'font-family': 'Roboto'}),
                dcc.Dropdown(
                id='sex-dropdown',
                options=[{'label': 'Female', 'value': 'F'}, {'label': 'Male', 'value': 'M'}],
                value=None,
                style={'backgroundColor': 'white', 'color': 'black'}
            ),  
            ], style={'width': '20%', 'display': 'inline-block', 'padding': '10px', 'backgroundColor': 'rgba(0,0,0,0)'}),
        ], style={'display': 'flex', 'justifyContent': 'center'}),  # Center the dropdown menus
    ]),
    dcc.Graph(id='crime-map')  # Define the graph where the map will be displayed
], style={'background': 'black'})  # Black background

# Define callback to update map based on dropdown selections
@app.callback(
    dash.dependencies.Output('crime-map', 'figure'),
    [dash.dependencies.Input('age-dropdown', 'value'),
     dash.dependencies.Input('borough-dropdown', 'value'),
     dash.dependencies.Input('sex-dropdown', 'value')]
)
def update_map(selected_age, selected_borough, selected_sex):
    filtered_df = df.copy()  # Create a copy of the dataframe to avoid modifying the original

    # Filter the dataframe based on the selected values from the dropdown menus
    if selected_age:
        filtered_df = filtered_df[filtered_df['Age'] == selected_age]
    if selected_borough:
        filtered_df = filtered_df[filtered_df['Borough'] == selected_borough]
    if selected_sex:
        filtered_df = filtered_df[filtered_df['Sex'] == selected_sex]

    # Create the map figure
    fig = px.scatter_mapbox(filtered_df, lat="Latitude", lon="Longitude",
                            color="Offense",
                            color_discrete_map=color_map,
                            custom_data=["Sex", "Borough", "Age", "Offense", "Color"],
                            zoom=10, height=600,
                            opacity=0.8)

    # Update the hover template and hover label
    fig.update_traces(
        hovertemplate="<br>".join([
            "Offense: %{customdata[3]}",
            "Sex: %{customdata[0]}",
            "Borough: %{customdata[1]}",
            "Age: %{customdata[2]}",
        ]),
        hoverlabel_bgcolor=filtered_df['Color'],  # Use the color column for the hover background color
        hoverlabel_font_color='white'  # Set the hover text color to white
    )

    # Update the layout of the figure
    fig.update_layout(
        mapbox_style="carto-positron",
        mapbox_zoom=10,
        mapbox_center={"lat": filtered_df["Latitude"].mean(), 
                       "lon": filtered_df["Longitude"].mean()},
        paper_bgcolor='rgba(0,0,0,0)',  # Set paper background to transparent
        plot_bgcolor='rgba(0,0,0,0)',  # Transparent background
        font=dict(  # Increase the size of the text inside the graph
            family="Roboto",
            size=18,
            color="white"
        ),
        legend=dict(
            itemsizing='constant',  # Make all legend items the same size
            itemwidth=30,  # Increase the width of the color dot
            font=dict(color='white')  # Change the color of the legend text to white
        ),
        hoverlabel=dict(  # Change the color of the hover information to white
            bgcolor="black", 
            font_size=16, 
            font_family="Roboto",
            font_color="white"
        )
    )
    return fig  # Return the figure to be displayed in the 'crime-map' graph

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True, port=8050)  # Run the app on port 8050 with debug mode on

### Visualizing NYPD Hate Crime Dataset
This Dash application visualizes NYPD Hate Crimes data. It includes a line chart showing the total number of hate crimes per year. When a point corresponding to a year on the line chart is clicked, two pie charts are updated to show the distribution of offense categories and law codes for that selected year. This allows for a dynamic and interactive exploration of the data.

In [11]:
# Import necessary libraries
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go

# Load the dataset
df = pd.read_csv("NYPD_Hate_Crimes.csv")

# Check for missing values
missing_values = df.isnull().sum()

# Fill missing values
for column in df.columns:
    if df[column].dtype == 'object':
        df[column].fillna(df[column].mode()[0], inplace=True)
    elif df[column].dtype in ['int64', 'float64']:
        df[column].fillna(df[column].mean(), inplace=True)

# Convert 'Record Create Date' and 'Arrest Date' to datetime format
df['Record Create Date'] = pd.to_datetime(df['Record Create Date'])
df['Arrest Date'] = pd.to_datetime(df['Arrest Date'])

# Grouping hate crimes by year
yearly_counts = df.groupby('Complaint Year Number').size().reset_index(name='Count')

# Custom color gradient
colors = ['#08519c', '#3182bd', '#6baed6', '#99d8c9', '#c7e9c0', '#006d2c']

# Initialize Dash app
app = dash.Dash(__name__)

# Define the layout of the app
app.layout = html.Div([
    dcc.Graph(id='crime-line-chart', style={'height': '50vh'}),
    html.Div([
        dcc.Graph(id='offense-category-pie-chart', style={'height': '50vh', 'width': '50%', 'display': 'inline-block'}),
        dcc.Graph(id='law-code-pie-chart', style={'height': '50vh', 'width': '50%', 'display': 'inline-block'})
    ], style={'backgroundColor': 'black'})
])

# Define callback for updating pie charts based on selected year
@app.callback(
    [Output('offense-category-pie-chart', 'figure'),
     Output('law-code-pie-chart', 'figure')],
    [Input('crime-line-chart', 'clickData')]
)
def update_pie_charts(clickData):
    if clickData is None:
        return go.Figure(), go.Figure()
    else:
        year = clickData['points'][0]['x']
        data_year = df[df['Complaint Year Number'] == year]
        offense_category_counts = data_year['Offense Category'].value_counts()
        law_code_counts = data_year['Law Code Category Description'].value_counts()

        # Convert the index to title case
        offense_category_counts.index = offense_category_counts.index.str.title()
        law_code_counts.index = law_code_counts.index.str.title()

        # Create pie charts for offense category and law code distribution
        fig1 = px.pie(values=offense_category_counts.values, names=offense_category_counts.index, title=f'<b>Offense Category Distribution for {year}</b>', color_discrete_sequence=colors)
        fig2 = px.pie(values=law_code_counts.values, names=law_code_counts.index, title=f'<b>Law Code Distribution for {year}</b>', color_discrete_sequence=colors)
        
        # Update layout for pie charts
        fig1.update_layout(paper_bgcolor='black', plot_bgcolor='black', font_color='white', legend=dict(font=dict(size=13), x=1.1, y=1))
        fig2.update_layout(paper_bgcolor='black', plot_bgcolor='black', font_color='white', legend=dict(font=dict(size=13), x=0.75, y=1))
        
        return fig1, fig2

# Define callback for updating line chart
@app.callback(
    Output('crime-line-chart', 'figure'),
    [Input('offense-category-pie-chart', 'figure'),
     Input('law-code-pie-chart', 'figure')]
)
def update_line_chart(fig1, fig2):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=yearly_counts['Complaint Year Number'], 
                             y=yearly_counts['Count'], 
                             mode='lines', 
                             line=dict(width=4),
                             hovertemplate='Year: %{x}<br>Arrest Count: %{y}<extra></extra>'))
    
    # Update layout for line chart
    fig.update_layout(title='<b>Hate Crimes Data Visualization</b>', 
                      title_x=0.5,  # this line centralizes the title
                      paper_bgcolor='black', 
                      plot_bgcolor='black', 
                      font_color='white', 
                      xaxis_title='Year', 
                      yaxis_title='Arrest Count',
                      xaxis=dict(showgrid=False, linecolor='white', linewidth=2),
                      yaxis=dict(showgrid=False, linecolor='white', linewidth=2))
    
    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True, port=8052)

### Comparing NYPD Arrest and Hate Crime Datasets
This Dash application visualizes NYPD Arrest and Hate Crime data in a dual-axis line chart, comparing the number of arrests and hate crimes over time. When a data point on the line chart is clicked, detailed information about the arrests or hate crimes on that date is displayed below the chart in the form of a pie chart (showing the distribution across boroughs/counties) and a bar chart (showing the distribution across offense categories). This interaction allows users to explore the data in a more detailed and dynamic way.

In [13]:
# Import necessary libraries
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import pandas as pd

# Load the datasets
arrest_df = pd.read_csv("NYPD_Arrest_Data.csv")
hate_crimes_df = pd.read_csv("NYPD_Hate_Crimes.csv")

# Convert the 'Record Create Date' and 'ARREST_DATE' to datetime
hate_crimes_df['Record Create Date'] = pd.to_datetime(hate_crimes_df['Record Create Date'])
arrest_df['ARREST_DATE'] = pd.to_datetime(arrest_df['ARREST_DATE'])

# Filter the data to only include records from January 2023 to December 2023
start_date = pd.to_datetime('2023-01-01')
end_date = pd.to_datetime('2023-12-12')
hate_crimes_df = hate_crimes_df[(hate_crimes_df['Record Create Date'] >= start_date) & (hate_crimes_df['Record Create Date'] <= end_date)]
arrest_df = arrest_df[(arrest_df['ARREST_DATE'] >= start_date) & (arrest_df['ARREST_DATE'] <= end_date)]

# Group the data by date and count the number of arrests and hate crimes
arrest_counts = arrest_df.groupby(arrest_df['ARREST_DATE'].dt.date).size()
hate_crime_counts = hate_crimes_df.groupby(hate_crimes_df['Record Create Date'].dt.date).size()

# Initialize the Dash app
app = dash.Dash(__name__)

# Define the layout of the app
app.layout = html.Div([
    dcc.Graph(
        id='main-plot',
        figure={
            'data': [
                # Plot the arrest counts
                go.Scatter(
                    x=arrest_counts.index,
                    y=arrest_counts.values,
                    mode='lines',
                    name='Arrests',
                    line=dict(color='#1e90ff'),  # Dodger Blue
                    yaxis='y1',
                    hovertemplate='<br>'.join([
                        "Date: %{x}",
                        "Year: %{x|%Y}",
                        "Type: Arrests",
                        "Count: %{y}"
                    ])
                ),
                # Plot the hate crime counts
                go.Scatter(
                    x=hate_crime_counts.index,
                    y=hate_crime_counts.values,
                    mode='lines',
                    name='Hate Crimes',
                    line=dict(color='#ffd700'),  # Gold
                    yaxis='y2',
                    hovertemplate='<br>'.join([
                        "Date: %{x}",
                        "Year: %{x|%Y}",
                        "Type: Hate Crimes",
                        "Count: %{y}"
                    ])
                )
            ],
            'layout': go.Layout(
                title='Comparison of Arrests and Hate Crimes Over Time (January 2023 - December 2023)',
                xaxis=dict(title='Date', color='white', linecolor='white', linewidth=2),
                yaxis=dict(
                    title='Number of Arrests',
                    titlefont=dict(color='#1e90ff'),  # Dodger Blue
                    tickfont=dict(color='white'),
                    linecolor='white',
                    linewidth=2
                ),
                yaxis2=dict(
                    title='Number of Hate Crimes',
                    titlefont=dict(color='#ffd700'),  # Gold
                    tickfont=dict(color='white'),
                    overlaying='y',
                    side='right',
                    linecolor='white',
                    linewidth=2
                ),
                showlegend=True,
                plot_bgcolor='black',
                paper_bgcolor='black',
                font=dict(color='white')
            )
        },
        clickData=None
    ),
    html.Div(id='details')
])

# Define the callback function to update the details section based on the clicked data point
@app.callback(
    Output('details', 'children'),
    [Input('main-plot', 'clickData')]
)
def update_details(clickData):
    if clickData is None:
        return ''

    date = clickData['points'][0]['x']
    data_type = clickData['points'][0]['curveNumber']

    # If the clicked data point is from the arrest dataset
    if data_type == 0:
        details = arrest_df[arrest_df['ARREST_DATE'] == date]
        borough_counts = details['ARREST_BORO'].value_counts()
        borough_counts.index = borough_counts.index.map({'K': 'Brooklyn', 'B': 'Bronx', 'M': 'Manhattan', 'Q': 'Queens', 'S': 'Staten Island'})
        offense_counts = details['OFNS_DESC'].value_counts()
        dataset = 'Arrest Dataset'
    # If the clicked data point is from the hate crime dataset
    else:
        details = hate_crimes_df[hate_crimes_df['Record Create Date'] == date]
        borough_counts = details['County'].value_counts()
        offense_counts = details['Offense Description'].value_counts()
        dataset = 'Hate Crime Dataset'

    colors = ['#08519c', '#3182bd', '#6baed6', '#99d8c9', '#c7e9c0', '#006d2c']

    # Create a pie chart for the borough counts
    borough_figure = go.Figure(data=[
        go.Pie(labels=borough_counts.index, values=borough_counts.values, hole=0.4, marker=dict(colors=colors))  # Donut chart with gradient colors
    ])
    borough_figure.update_layout(
        title_text='Boroughs/Counties for ' + date,
        plot_bgcolor='black',
        paper_bgcolor='black',
        font=dict(color='white')
    )

    # Create a bar chart for the offense counts
    offense_figure = go.Figure(data=[
        go.Bar(name='Offenses', x=offense_counts.index, y=offense_counts.values, marker_color='#99d8c9')  # Changed color
    ])
    offense_figure.update_layout(
        title_text='Offense Categories for ' + date,
        plot_bgcolor='black',
        paper_bgcolor='black',
        font=dict(color='white'),
        xaxis=dict(color='white'),
        yaxis=dict(color='white')
    )
    return html.Div([
        html.H3('Data point chosen from: ' + dataset, style={'font-family': 'Arial', 'font-size': '16px', 'color':'white', 'margin-bottom': '0px'}),
        html.Div([
            dcc.Graph(figure=borough_figure, style={'display': 'inline-block', 'width': '40%'}),
            dcc.Graph(figure=offense_figure, style={'display': 'inline-block', 'width': '60%'})
        ], style={'backgroundColor': 'black', 'margin': '0px'})
    ], style={'backgroundColor': 'black', 'margin': '0px'})

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True, port=8053)